**Find Trends in the data for modeling forward pass**

Similar to `experiments/step_time_analysis.ipynb` we are using current batch data we gathered in profiling to estimate forward pass times with a model

In [17]:
from utils import parse_csv, format_data, concatenate_dataframes # found in utils.py

path1 = './profiling/train'

# # parse and concat all dataframes
df = concatenate_dataframes(path1)
df = format_data(df)
# df.to_csv('formatted_execution_stats500.csv', index=False)

In [20]:
len(df)

170050

In [18]:
# try to find correlation between execute_time and num_decodes, num_prefills, sum_decode_tokens, sum_prefill_tokens
correlation = df[['execute_time', 'num_prefills', 'sum_decode_tokens', 'sum_prefill_tokens', 'max_prefill_tokens']].corr()
print(correlation)

                    execute_time  num_prefills  sum_decode_tokens  \
execute_time            1.000000      0.758636           0.741600   
num_prefills            0.758636      1.000000           0.448262   
sum_decode_tokens       0.741600      0.448262           1.000000   
sum_prefill_tokens      0.716647      0.566984           0.258445   
max_prefill_tokens      0.689020      0.539195           0.248068   

                    sum_prefill_tokens  max_prefill_tokens  
execute_time                  0.716647            0.689020  
num_prefills                  0.566984            0.539195  
sum_decode_tokens             0.258445            0.248068  
sum_prefill_tokens            1.000000            0.982670  
max_prefill_tokens            0.982670            1.000000  


In [ ]:
# train a linear regression model to predict execute_time based on num_prefills, sum_decode_tokens, sum_prefill_tokens, and arrival_rate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X = df[['sum_decode_tokens', 'sum_prefill_tokens', 'max_prefill_tokens','num_prefills']]
y = df['execute_time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)
print("Score:", model.score(X_test, y_test))
# make predictions on the test set

Coefficients: [ 6.26600355e+01  3.01297950e-02 -1.08762510e-02  2.64417673e+03]
Intercept: 4927.46099843045
Score: 0.9174653252856592
